In [36]:
import wrangle
from env import host, user, password
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
url = f'mysql+pymysql://{user}:{password}@{host}iris_db'
from pydataset import data

import env
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from env import host, user, password
import sys
from sklearn.impute import SimpleImputer
import split_scale
import explore
import acquire
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import acquire
from pydataset import data

Gather

In [63]:
from pydataset import data

iris = data('iris')
iris
print(iris.head(3))
df = iris

   Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
1           5.1          3.5           1.4          0.2  setosa
2           4.9          3.0           1.4          0.2  setosa
3           4.7          3.2           1.3          0.2  setosa


In [65]:
X = df[['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width']]
y = df[['Species']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

X_train.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
115,5.8,2.8,5.1,2.4
137,6.3,3.4,5.6,2.4
54,5.5,2.3,4.0,1.3
20,5.1,3.8,1.5,0.3
39,4.4,3.0,1.3,0.2


In [66]:
y_train.head()

,Species
115,virginica
137,virginica
54,versicolor
20,setosa
39,setosa


Create Basic Model

Create model object

Fit the model to the data

Predict labels

Estimate probability of a label estimate

In [67]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')

In [24]:
y_train.head()

,Species
115,virginica
137,virginica
54,versicolor
20,setosa
39,setosa


Create the logistic regression object

In [25]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver= 'newton-cg')
#logit.fit(X_train, y_train)

Fit the model to the training data

In [68]:
logit.fit(X_train, y_train)

TypeError: '<' not supported between instances of 'str' and 'int'

In [17]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 []
Intercept: 
 [0. 0. 0.]


KeyError: "['species_id' 'measurement_id'] not found in axis"